In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 11 12:29:48 2019

@author: SLab
"""
import tensorflow as tf
import keras.callbacks
#import sys
import os
import pandas
from keras import backend as K
from keras.utils.vis_utils import plot_model
from keras import optimizers
from keras.models import Sequential
from keras.layers import  Dropout
#import keras
from scipy import interp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas
import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from subprocess import check_output
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
#from sklearn import cross_validation

import os, re, csv, math, codecs
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve
import time
from sklearn.metrics import roc_curve

from sklearn.metrics import auc
from sklearn.metrics import average_precision_score

Using TensorFlow backend.


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Helping functions

In [6]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))

def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc


from sklearn.metrics import roc_auc_score
def auroc(y_true, y_pred):
    return roc_auc_score(y_true, y_pred)

# auc = roc_auc_score(testy, yhat_probs)

from sklearn.metrics import cohen_kappa_score
def kappa(y_true, y_pred):
    cohen_kappa_score(y_true, y_pred)



In [7]:
sns.set_style("whitegrid")
np.random.seed(0)

DATA_PATH = '../input/'
EMBEDDING_DIR = '../input/'

MAX_NB_WORDS = 100000
tokenizer = RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))
stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])

class LossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.accuracies = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.accuracies.append(logs.get('acc'))
        
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.times = []

    def on_epoch_begin(self, epoch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, epoch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)
        
#print(check_output(["ls", "../input"]).decode("utf8"))

print('loading word embeddings...')
embeddings_index = {}
f = codecs.open('./Datasets/citation_data/wiki.simple.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('found %s word vectors' % len(embeddings_index))


train_df = pd.read_csv('./Datasets/citation_data' + '/final_data_citations_B.csv', sep=',', header=0,encoding = "ISO-8859-1")
test_df = pd.read_csv('./Datasets/citation_data' + '/final_data_citations_B_test.csv', sep=',', header=0,encoding = "ISO-8859-1")
test_df = test_df.fillna('_NA_')

print("num train: ", train_df.shape[0])
print("num test: ", test_df.shape[0])

label_names = ["positional", "essential"]
y_train = train_df[label_names].values

#visualize word distribution
train_df['doc_len'] = train_df['text'].apply(lambda words: len(words.split(" ")))
max_seq_len = np.round(train_df['doc_len'].mean() + train_df['doc_len'].std()).astype(int)
#print("####################################################")
#print(max_seq_len)
#==============================================================================
# sns.distplot(train_df['doc_len'], hist=True, kde=True, color='b', label='doc len')
# print("####################################################")
# plt.axvline(x=max_seq_len, color='k', linestyle='--', label='max len')
# plt.title('comment length'); plt.legend()
# plt.show()
#==============================================================================
raw_docs_train = train_df['text'].tolist()
raw_docs_test = test_df['text'].tolist() 
num_classes = len(label_names)

print("pre-processing train data...")
processed_docs_train = []
for doc in tqdm(raw_docs_train):
    tokens = tokenizer.tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_train.append(" ".join(filtered))
#end for

processed_docs_test = []
for doc in tqdm(raw_docs_test):
    tokens = tokenizer.tokenize(doc)
    filtered = [word for word in tokens if word not in stop_words]
    processed_docs_test.append(" ".join(filtered))
#end for

print("tokenizing input data...")
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(processed_docs_train + processed_docs_test)  #leaky
word_seq_train = tokenizer.texts_to_sequences(processed_docs_train)
word_seq_test = tokenizer.texts_to_sequences(processed_docs_test)
word_index = tokenizer.word_index
print("dictionary size: ", len(word_index))

#pad sequences
word_seq_train = sequence.pad_sequences(word_seq_train, maxlen=max_seq_len)
word_seq_test = sequence.pad_sequences(word_seq_test, maxlen=max_seq_len)

random_state = np.random.RandomState(0)
X_train, X_test, Y_train, Y_test = train_test_split(word_seq_train, y_train, test_size=.2,
                                                    random_state=random_state)

#print (X_test)
#print("########################")
#print(Y_test)

batch_size = 256 
num_epochs = 1 

#model parameters
num_filters = 64 
embed_dim = 300 
weight_decay = 1e-3

#embedding matrix
print('preparing embedding matrix...')
words_not_found = []
nb_words = min(MAX_NB_WORDS, len(word_index)+2)
embedding_matrix = np.zeros((nb_words, embed_dim))
for word, i in word_index.items():
    if i >= nb_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

print("sample words not found: ", np.random.choice(words_not_found, 10))

print("training CNN ...")
model = Sequential()
model.add(Embedding(nb_words, embed_dim,
          weights=[embedding_matrix], input_length=max_seq_len, trainable=False))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(num_classes, activation='sigmoid'))  #multi-label (k-hot encoding)

class_weight = {0: 1,
                1: 9}
#class_weight=class_weight
history = LossHistory()
time_callback = TimeHistory()
#metrics = Metrics()
#keras.optimizers.Adam(lr=0.001)
adam = optimizers.Adam(lr=0.0001)
#, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy',precision,recall, f1])
model.summary()
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]
pandas.DataFrame( model.fit(X_train, Y_train, validation_data=(X_test, Y_test), 
                                  batch_size=256,class_weight=class_weight, epochs=100, shuffle=True, verbose=2, callbacks=[history,time_callback]).history).to_csv("fullData-ClassWeight0.5-Adam_B_001.csv")

print("Binaryclass-fullData-classweight-done")


443it [00:00, 4401.31it/s]

loading word embeddings...


111052it [00:20, 5490.88it/s]


found 111052 word vectors
num train:  9518
num test:  520


  5%|███▌                                                                         | 435/9518 [00:00<00:02, 4322.87it/s]

pre-processing train data...


100%|█████████████████████████████████████████████████████████████████████████████| 520/520 [00:00<00:00, 55392.85it/s]


tokenizing input data...
dictionary size:  25519
preparing embedding matrix...
number of null word embeddings: 12772
sample words not found:  ['gottron' 'event2' 'planarity' 'stipulations' 'generalizeâ' 'diminum'
 'schuhmann' 'binot' 'it13' 'entence']
training CNN ...
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 213, 300)          7656300   
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 213, 64)           134464    
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 106, 64)           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 106, 64)           28736     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 64)          

Epoch 33/100
 - 3s - loss: 0.3465 - accuracy: 0.9498 - precision: 0.9484 - recall: 0.9515 - f1: 0.9499 - val_loss: 0.3276 - val_accuracy: 0.8860 - val_precision: 0.8831 - val_recall: 0.8774 - val_f1: 0.8802
Epoch 34/100
 - 3s - loss: 0.3391 - accuracy: 0.9512 - precision: 0.9503 - recall: 0.9522 - f1: 0.9513 - val_loss: 0.3150 - val_accuracy: 0.8957 - val_precision: 0.8918 - val_recall: 0.8882 - val_f1: 0.8900
Epoch 35/100
 - 3s - loss: 0.3216 - accuracy: 0.9522 - precision: 0.9513 - recall: 0.9535 - f1: 0.9524 - val_loss: 0.3188 - val_accuracy: 0.8921 - val_precision: 0.8888 - val_recall: 0.8826 - val_f1: 0.8857
Epoch 36/100
 - 3s - loss: 0.3038 - accuracy: 0.9561 - precision: 0.9547 - recall: 0.9576 - f1: 0.9561 - val_loss: 0.3234 - val_accuracy: 0.8892 - val_precision: 0.8850 - val_recall: 0.8822 - val_f1: 0.8836
Epoch 37/100
 - 3s - loss: 0.2934 - accuracy: 0.9588 - precision: 0.9578 - recall: 0.9604 - f1: 0.9591 - val_loss: 0.3136 - val_accuracy: 0.8968 - val_precision: 0.8932 - v

Epoch 73/100
 - 3s - loss: 0.1114 - accuracy: 0.9922 - precision: 0.9920 - recall: 0.9924 - f1: 0.9922 - val_loss: 0.3535 - val_accuracy: 0.9126 - val_precision: 0.9070 - val_recall: 0.9066 - val_f1: 0.9068
Epoch 74/100
 - 3s - loss: 0.1035 - accuracy: 0.9935 - precision: 0.9937 - recall: 0.9934 - f1: 0.9935 - val_loss: 0.3638 - val_accuracy: 0.9133 - val_precision: 0.9075 - val_recall: 0.9075 - val_f1: 0.9075
Epoch 75/100
 - 3s - loss: 0.1023 - accuracy: 0.9923 - precision: 0.9922 - recall: 0.9922 - f1: 0.9922 - val_loss: 0.3558 - val_accuracy: 0.9115 - val_precision: 0.9061 - val_recall: 0.9056 - val_f1: 0.9058
Epoch 76/100
 - 3s - loss: 0.1018 - accuracy: 0.9924 - precision: 0.9923 - recall: 0.9926 - f1: 0.9924 - val_loss: 0.3656 - val_accuracy: 0.9128 - val_precision: 0.9071 - val_recall: 0.9071 - val_f1: 0.9071
Epoch 77/100
 - 3s - loss: 0.0980 - accuracy: 0.9935 - precision: 0.9935 - recall: 0.9934 - f1: 0.9935 - val_loss: 0.3572 - val_accuracy: 0.9099 - val_precision: 0.9040 - v

In [8]:
# # ==============================================================================
# pandas.DataFrame(model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=30,
#           epochs=100, shuffle=True, callbacks=[earlystop_cb, check_cb, history]).history).to_csv("history.csv")
# # ==============================================================================
# # ==============================================================================
# # ==============================================================================
# y_test = model.predict(word_seq_test)
# submission_df = pd.DataFrame(columns=['id'] + label_names)
# submission_df['id'] = test_df['id'].values 
# submission_df[label_names] = y_test 
# submission_df.to_csv("./cnn_fasttext_submission.csv", index=False)
# plt.figure()
# # ==============================================================================
# # ==============================================================================
# # ==============================================================================
# plt.plot(hist.history['loss'], lw=2.0, color='b', label='train')
# plt.plot(hist.history['val_loss'], lw=2.0, color='r', label='val')
# plt.title('CNN sentiment')
# plt.xlabel('Epochs')
# plt.ylabel('Cross-Entropy Loss')
# plt.legend(loc='upper right')
# plt.show()
# plt.figure()
# plt.plot(hist.history['acc'], lw=2.0, color='b', label='train')
# plt.plot(hist.history['val_acc'], lw=2.0, color='r', label='val')
# plt.title('CNN sentiment')
# plt.xlabel('Epochs')
# plt.ylabel('Accuracy')
# plt.legend(loc='upper left')
# plt.show()
# # ==============================================================================


NameError: name 'y_test' is not defined

# Class Metrics
commented

In [6]:
#==============================================================================
# class Metrics(keras.callbacks.Callback):
#     def on_train_begin(self, logs={}):
#         self.confusion = []
#         self.precision = []
#         self.recall = []
#         self.f1s = []
#         self.kappa = []
#         self.auc = []
# 
#     def on_epoch_end(self, epoch, logs={}):
#       score = np.asarray(self.model.predict(self.validation_data[0]))
#       predict = np.round(np.asarray(self.model.predict(self.validation_data[0])))
#       predict2 = np.asarray(self.model.predict(self.validation_data[0]))
#       #print("#######y_predict2s#####")
#       #print(predict2)
#       #print("#######y_predict#####")
#       #print(predict)
#       targ = self.validation_data[1]
#       #print("#######targ#####")
#       #print(targ)
#       y_test_non_category = [ np.argmax(t) for t in targ ]
#       y_predict_non_category = [ np.argmax(t) for t in predict ]
#       y_predict_non_category2 = [ np.amax(t) for t in predict2 ]
#       #print("#######y_predict_non_category2#####")
#       #print(y_predict_non_category2)
#       #print("#######y_test_non_category#####")
#       #print(y_test_non_category)
#       conf_mat = confusion_matrix(y_test_non_category, y_predict_non_category)
#       print("########confusion_matrics########")
#       print(conf_mat)
#         #self.auc.append(sklm.roc_auc_score(targ, score))
# #==============================================================================
# #       self.confusion.append(confusion_matrix(y_test_non_category, y_predict_non_category))
# #       print("confusion_matrics 2")
# #       print(self.confusion)
# #       
# #==============================================================================
#       report = classification_report(y_test_non_category,  y_predict_non_category, target_names=label_names)
#       print("########classification report########")
#       print(report) 
#       
#       fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test_non_category,  y_predict_non_category)
#       auc_keras = auc(fpr_keras, tpr_keras)
#       print("########ROC##########")
#       print(auc_keras) 
#       #self.classification_report.append(classification_report(y_test_non_category,  y_predict_non_category, target_names=label_names))
#       self.precision.append(precision_score(y_test_non_category, y_predict_non_category,average='weighted'))
#       self.recall.append(recall_score(y_test_non_category, y_predict_non_category,average='weighted'))
#       #auc_PRC = auc(self.recall, self.precision)
# #==============================================================================
#       precision1, recall1, thresholds1 = precision_recall_curve(y_test_non_category, y_predict_non_category)
#       print("########PRC##########")
#       auc_PRC= auc(recall1, precision1)
#       print(auc_PRC)
# #==============================================================================
#  # Compute ROC curve and ROC area for each class
#       fpr = dict()
#       tpr = dict()
#       roc_auc = dict()
#       n_classes=2
#       for i in range(n_classes):
#           fpr[i], tpr[i], _ = roc_curve(targ[:, i], predict[:, i])
#           roc_auc[i] = auc(fpr[i], tpr[i])
# 
# # Compute micro-average ROC curve and ROC area
#       fpr["micro"], tpr["micro"], _ = roc_curve(targ.ravel(), predict.ravel())
#       roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
# 
# # Compute macro-average ROC curve and ROC area
# 
# # First aggregate all false positive rates
#       all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
# 
# # Then interpolate all ROC curves at this points
#       mean_tpr = np.zeros_like(all_fpr)
#       for i in range(n_classes):
#           mean_tpr += interp(all_fpr, fpr[i], tpr[i])
# 
# # Finally average it and compute AUC
#       mean_tpr /= n_classes
# 
#       fpr["macro"] = all_fpr
#       tpr["macro"] = mean_tpr
#       roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
#       print("########ROC_MAC##########")
#       print(roc_auc) 
#       n_classes=2
#       precision = dict()
#       recall = dict()
#       average_precision = dict()
#       for i in range(n_classes):
#             precision[i], recall[i], _ = precision_recall_curve(targ[:, i],
#                                                                 predict[:, i])
#             average_precision[i] = average_precision_score(targ[:, i], predict[:, i])
#         
#         # A "micro-average": quantifying score on all classes jointly
#       precision["micro"], recall["micro"], _ = precision_recall_curve(targ.ravel(),
#             predict.ravel())
#       average_precision["macro"] = average_precision_score(targ, predict,
#                                                              average="macro")
#       average_precision["micro"] = average_precision_score(targ, predict,
#                                                              average="micro")
#       average_precision["weighted"] = average_precision_score(targ, predict,
#                                                              average="weighted")
#       print('Average precision score, Macro-averaged over all classes: {0:0.5f}' .format(average_precision["macro"]))
#       print('Average precision score, Micro-averaged over all classes: {0:0.5f}' .format(average_precision["micro"]))
#       print('Average precision score, Weighted-averaged over all classes: {0:0.5f}' .format(average_precision["weighted"]))
#       
#       return
#==============================================================================